# Managing external access to services in a cluster
A good way to facilitate and organize the entry of information and access to cluster is by creating an [Ingress](https://kubernetes.io/docs/concepts/services-networking/ingress/).
> "A ingress is an API object that manages external access to the services in a cluster, typically HTTP. Ingress may provide load balancing, SSL termination and name-based virtual hosting."

Ingresses allows you to route and expose acess and traffic through the cluster to external acess under a single ip adress. To make a ingress to work you must have:

1. Services to organize and route (we will use some basic examples)
2. An Ingress controller (k3s have the traefik ingress controller set up by default)

After getting this requirements you can apply a manifest file with the instructions to route the incomming requests to they respective services. This notebook is based on the [kubernetes documentation](https://kubernetes.io/docs/concepts/services-networking/ingress/) and in this [simple tutorial](https://matthewpalmer.net/kubernetes-app-developer/articles/kubernetes-ingress-guide-nginx-example.html) about how to configure a simple Ingress to acess many services.

##  1 Creating basics servers and services

First let's create two simple applications, an nginx and an flask servers, and their respective services. We will be using the manifest files in the "issue7" folder to make the process faster.

In [30]:
import pykube
import yaml,json

api = pykube.HTTPClient(pykube.KubeConfig.from_file("k3s.yaml"))

nginx_deployment = open("issue7/nginx/deployment.yaml", "r")
nginx_service = open("issue7/nginx/service.yaml", "r")
flask_deployment = open("issue7/flask/deployment.yaml", "r")
flask_service = open("issue7/flask/service.yaml", "r")

specs = []

specs.append(yaml.load(nginx_deployment.read(), Loader=yaml.FullLoader))
specs.append(yaml.load(nginx_service.read(), Loader=yaml.FullLoader))
specs.append(yaml.load(flask_deployment.read(), Loader=yaml.FullLoader))
specs.append(yaml.load(flask_service.read(), Loader=yaml.FullLoader))

for spec in specs:
    if spec['kind'] == 'Service':
        pykube.Service(api,spec).create()
    elif spec['kind'] == 'Deployment':
        pykube.Deployment(api,spec).create()
    
nginx_deployment.close()
nginx_service.close()
flask_deployment.close()
flask_service.close()

## 2 Configuring the Ingress
Once the k3s cluster already have the traefik ingress controller we don't need to set up this. We can step up to the Ingress specification and test. The code below describe the rules for a Ingress to route the incomming requests on the localhost to "flask" and "nginx" services.

In [31]:
ingress = {
  "apiVersion": "extensions/v1beta1", 
  "kind": "Ingress", 
  "metadata": {
    "name": "ingress-example", 
    "annotations": {
      "ingress.kubernetes.io/rewrite-target": "/"
    }
  },
  "spec": {
    "rules": [
      {
        "http": {
          "paths": [
            {
              "path": "/nginx", 
              "backend": {
                "serviceName": "nginx", 
                "servicePort": 8000
              }
            }, 
            {
              "path": "/flask", 
              "backend": {
                "serviceName": "flask", 
                "servicePort": 5000
              }
            }
          ]
        }
      }
    ]
  }, 
}
pykube.Ingress(api,ingress).create()

Notice the annotation [ingress.kubernetes.io/rewrite-target](https://kubernetes.github.io/ingress-nginx/user-guide/nginx-configuration/annotations/#rewrite), it says to the ingress to redirect the request to the "/" path of the services.

You now can test the ingress by acessing [localhost/flask](http://localhost/flask) or [localhost/nginx](http://localhost/nginx) urls.
> 1. You can also acess nginx error file by specifyng the 50x.html path in the request [localhost/nginx/50x.html](http://localhost/nginx/50x.html)
> 2. If you try to acess a [wrong path](http://localhost/flask/wrong-url) in the flask application it will show the flask app error message instead the [nginx error](http://localhost/nginx/wrong-path) message.

## 3 Clean up the cluster
Run the below cell to delete the services, deployments and the ingress created.

In [32]:
for spec in specs:
    if spec['kind'] == 'Service':
        pykube.Service(api,spec).delete()
    elif spec['kind'] == 'Deployment':
        pykube.Deployment(api,spec).delete()
pykube.Ingress(api,ingress).delete()

# 4 Using the ingress to expose the dask interface
One usefull thing to do with a Ingress is expose the dask's many interfaces of the scheduler and workers. Let's create a dask cluster with dask-kubernetes (like we did in Issue6) and try to expose his interface with an Ingress.

## 4.1 Create the dask cluster
Run the two cells bellow to create a dask single worker cluster and his client.

In [36]:
from dask.distributed import Client
from dask_kubernetes import KubeCluster

file = open("daskKubernetes/worker.yaml", 'r')
worker = yaml.load(file.read(), Loader=yaml.FullLoader)
file.close()

worker['spec']['volumes'] = [{'name':'storage','persistentVolumeClaim':{'claimName':'claim'}}]
worker['spec']['containers'][0]['volumeMounts'] = [{'name':'storage','mountPath':'/home/arkaisho'}]

cluster = KubeCluster.from_dict(worker)
cluster.scale(1)

distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:    tcp://10.42.0.11:46447
distributed.scheduler - INFO -   dashboard at:                     :8787


In [38]:
client = Client(cluster)
client

Client Scheduler: tcp://10.42.0.11:46447 Dashboard: http://10.42.0.11:8787/status,Cluster Workers: 1 Cores: 2 Memory: 4.00 GB


# 4.2 Create the service and the ingress
In order to make the ingress route to the right place, we must have an service exposing the application that we want to reach. In this case, we first have to expose the dask scheduler dashboard with one service (we will use the ClusterIP type to show that we can expose services that are only visible inside the cluster using the Ingress for outsiders access).

In [67]:
service = {
    "apiVersion": "v1",
    "kind": "Service",
    "metadata": {
        "name": "scheduler"
    },
    "spec": {
        "selector": {
            "app": "notebook"
        },
        "ports": [
            {
                "port": 8787,
                "targetPort": 8787
            }
        ],
        "type": "ClusterIP"
    }
}

pykube.Service(api,service).create()

Now we can point the ingress to the service.

In [ ]:
ingress = {
  "apiVersion": "extensions/v1beta1", 
  "kind": "Ingress", 
  "metadata": {
    "name": "scheduler-ingress", 
    "annotations": {
      "traefik.ingress.kubernetes.io/rewrite-target": "/"
    }
  },
  "spec": {
    "rules": [
      {
        "http": {
          "paths": [
            {
              "path": "/scheduler", 
              "backend": {
                "serviceName": "scheduler", 
                "servicePort": 8787
              }
            }
          ]
        }
      }
    ]
  }, 
}

pykube.Ingress(api,ingress).create()

Notice that we have rewrited the target url with "/" in order to request the right url to the service.

- If we access [localhost/scheduler](http://localhost/scheduler) it will redirect to the service calling "scheduler-ip:application-port/"

- So, it allow us to acess [localhost/scheduler/status](http://localhost/scheduler/status) and it will call the service with "application-ip:application-port/status".

We need to do this due two things:
1. Load the statics files of the application
> If we don't rewrite the "/scheduler", the site try to get "/static" but the Ingress will call "/scheduler/static" inside the service, wich doesn't exists.
2. Organize the ingress in order to have more than one application exposed by ingresses.
> We can organize many applications in the same IP address only by calling "ip-address/application-name/index" and it will redirect the request to the correct service and port.

## 4.3 Acess the Ingress Url and see the dashboard working
Now you can acess the ingress [dashboard](http://localhost/scheduler/status) url and then run the cells bellow to see it showing the processes running in the cluster.

In [68]:
from dask import delayed
import time

def inc(x):
    time.sleep(0.01)
    return x+1
def red(x):
    time.sleep(0.01)
    return x-1
a=1
for i in range(100):
    b=delayed(inc)(a)
    a=delayed(red)(b)

In [70]:
%%time
a.compute()

CPU times: user 6.73 s, sys: 82.1 ms, total: 6.82 s
Wall time: 6.87 s


1

# 5 Close and clean everything

In [71]:
client.close()
cluster.close()

pykube.Ingress(api,ingress).delete()
pykube.Service(api,service).delete()

distributed.scheduler - INFO - Remove client Client-3e7b4f7a-a825-11ea-8023-52f9387825f0
distributed.scheduler - INFO - Remove client Client-3e7b4f7a-a825-11ea-8023-52f9387825f0
distributed.scheduler - INFO - Close client connection: Client-3e7b4f7a-a825-11ea-8023-52f9387825f0
distributed.scheduler - INFO - Scheduler closing...
distributed.scheduler - INFO - Scheduler closing all comms
distributed.scheduler - INFO - Remove worker <Worker 'tcp://10.42.0.29:43861', name: 0, memory: 0, processing: 0>
distributed.core - INFO - Removing comms to tcp://10.42.0.29:43861
distributed.scheduler - INFO - Lost all workers
